In [1]:
# 포물선 운동 애니메이션
# 이동 애니메이션

import sys
import time
import math
import gym
import numpy as np
from PyQt5.QtCore import *
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

# 발사 여부를 계산할 신경망 생성
# 뉴런의 수와 은닉층의 수는 테스트를 진행하면서 변경해가면서 적정수를 찾는다

def create_fire_model():
    fire_model = Sequential()
    fire_model.add(Dense(64, activation='relu', input_shape=(2,)))
    fire_model.add(Dense(64, activation='relu'))
    fire_model.add(Dense(64, activation='relu'))
    fire_model.add(Dense(64, activation='relu'))
    fire_model.add(Dense(2, activation="softmax"))

    fire_model.compile(
        loss="categorical_crossentropy",
        optimizer="adam",
        metrics=["accuracy"])
    return fire_model

# 발사 각도(rad)를 계산할 신경망 생성
def create_rad_model():
    rad_model = Sequential()
    rad_model.add(Dense(64, activation='relu', input_shape=(2,)))
    rad_model.add(Dense(64, activation='relu'))
    rad_model.add(Dense(64, activation='relu'))
    rad_model.add(Dense(64, activation='relu'))
    rad_model.add(Dense(1, activation="linear"))

    rad_model.compile(
        loss="mse",
        optimizer="adam",
        metrics=["mae"])
    return rad_model

# 파일에 저장된 모델의 가중치를 로드하여 모델에 적용
action_model = create_fire_model()  # 모델생성 및 컴파일
action_model.load_weights('action_model_weights.h5')

rad_model = create_rad_model()
rad_model.load_weights('rad_model_weights.h5')

class Target:
    def __init__(self):
        self.x = 0
        self.y = 300
    def move(self):
        self.x += 1
        self.y = int((1/132) * (self.x-200)**2 + 0) # 왜 int 형변환해?
    def get_state(self):
        return self.x, self.y
    def reset(self):
        self.x = 0
        self.y = 300
    
class Bullet:
    def __init__(self, rad):
        self.rad = rad
        self.x = 195
        self.y = 290
        self.vx = math.cos(rad)*3
        self.vy = -math.sin(rad)*3    # sin이 -인 이유는 y좌표는 위로가야하기 때문에
        self.target_state = (0,0)
    def move(self):
        self.x += self.vx
        self.y += self.vy

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.target = Target()
        self.bullets = []
        self.label = QLabel()
        self.label.setAlignment(Qt.AlignCenter)
        self.label.setPixmap(QPixmap(400, 300))
        self.ishit = False   # 맞췄을때
        # Target이 0에서 오른쪽으로 400이동해야 하기때문에 에피소드 횟수를 400으로 함
        self.episodes = 400  
        self.fireCnt = 0
        self.hits = 0
        self.hit_state = []
        self.hit_rad = []
        self.timer = QTimer()
        self.start = 0
        self.rudder = 90
        self.skip = 10
        self.rudder_x = 0
        self.rudder_y = -280
        
        move_action = QAction('&Start', self)
        #move_action.setStatusTip("Click to start")
        move_action.triggered.connect(self.action_start)
        
        stop_action = QAction('Stop', self)
        stop_action.triggered.connect(self.action_stop)
        
        autosemi_action = QAction('Semi-auto',self)
        autosemi_action.triggered.connect(self.semiauto)
        
        auto_action = QAction('&Auto',self)
        auto_action.triggered.connect(self.action_auto)
        
        toolbar = QToolBar()
        self.addToolBar(toolbar)
        
        toolbar.addAction(move_action)
        toolbar.addAction(autosemi_action)
        toolbar.addAction(auto_action)
        toolbar.addAction(stop_action)

        self.setCentralWidget(self.label)
    
    def paintEvent(self, event):
        painter = QPainter(self.label.pixmap())
        painter.fillRect(QRect(0,0,400,300), Qt.black)  # 화면 지우기
        
        if self.start==1 :
            pen = QPen(Qt.white, 3, Qt.CustomDashLine)
            pen.setDashPattern([4, 3, 2, 5])
            painter.setPen(pen)
            painter.drawLine(200, 295, (200 + int(self.rudder_x)), (280 + int(self.rudder_y)) )
        
        #painter.setPen(QPen(Qt.green,  8, Qt.SolidLine))  # 테두리 선
        brush = None
        if self.ishit:
            brush = QBrush(Qt.white, Qt.SolidPattern) # 폭발시 흰색으로 타겟을 그림
            self.ishit = False
        else: brush = QBrush(Qt.red, Qt.SolidPattern)
        painter.setBrush(brush)  # 채울 색상, 패턴
        # drawEllipse(QRect), QRect(x, y, w, h)
        painter.drawEllipse(self.target.x, self.target.y, 20, 20)
        #Target의 x,y좌표에 원 그리기
        
        for b in self.bullets:  #총알을 하나하나 꺼낸다
            painter.setPen(Qt.white)
            painter.setBrush(QBrush(Qt.white, Qt.SolidPattern))
            painter.drawEllipse(int(b.x), int(b.y), 10, 10)
        painter.end()
        
    def set_models(self,actionM,radM):
        self.action_model = actionM
        self.rad_model = radM
        
    def action_move(self):
        self.timer = QTimer()
        self.timer.setInterval(30)
        self.timer.timeout.connect(self.move)
        self.timer.start()
        
    def action_start(self):
        self.start = 1
        
    def semiauto(self):
        self.start = 2
        
    def action_auto(self):
        self.start= 3
        done = False
        while not done:
            state = self.target.get_state()
            prob = action_model.predict(np.array(state).reshape(-1,2))
            action = np.argmax(prob[0])
            self.skip -= 1
            print(action)
            if self.skip!=0 :
                action = 0
            else :
                self.skip=10
            print(action)
            rad = rad_model.predict(np.array(state).reshape(-1,2))
            return action,rad

    def action_stop(self):
        self.timer.stop()
        
    def move(self):
        self.target.move()
        for b in self.bullets:
            b.move()
            #포탄의 y좌표가 천장에 닿았을때 or x좌표가 왼쪽,오른쪽에 닿앗을때
            if b.y<=-10 or b.x<=0 or b.x>=400:
                self.bullets.remove(b)
            # 타겟과 발사체의 충돌(명중) 검사
            # 피타고라스 정의를 사용
            if ((self.target.x-b.x)**2 + (self.target.y-b.y)**2) < (20**2) :
                #명중 했을때의 타겟위치의 상태
                self.hit_state.append(b.target_state)
                #명중 했을때의 타겟위치의 총알의 시작각도
                self.hit_rad.append(b.rad)
                try:
                    #명중시 그 탄환 삭제 위에서 remove된 총알을 또 삭제할려고 할
                    #때 오류가 발생함으로
                    # 그 오류를 그냥 pass하기 위해 try except를 사용함
                    # 또는 elif를 사용하면 문제가 사라짐
                    self.bullets.remove(b)
                except:
                    pass
                # ishit 맞췄을시에 Target을 흰색으로 표시하기 위한 변수
                self.ishit = True 
                self.hits += 1
        #좌표가 바뀔때 마다 화면을 다시그리기 위한 함수 repaint()
        self.repaint()

    def keyPressEvent(self, evt):
        if evt.key()==Qt.Key_Space:
            if self.start==1 :
                rad = math.radians(self.rudder)
                self.step(1,rad)
                
            elif self.start==2:
                while True:
                    state = self.target.get_state()
                    prob = action_model.predict(np.array(state).reshape(-1,2))
                    if np.argmax(prob[0])==1: #쏠때가 아니라면 쏘지 않는다
                        rad = rad_model.predict(np.array(state).reshape(-1,2))
                        self.step(1,rad)
                        break
        if evt.key() ==Qt.Key_Right:
            self.rudder -= 5
            print(self.rudder)                                                              # 탄환이 움직이는 각도
            self.rudder_x = math.cos(math.radians(self.rudder))*200
            self.rudder_y = -math.sin(math.radians(self.rudder))*200
#             self.rudder_y = -math.sin(3.14/2)*200    # 선 움직이기
        if evt.key()==Qt.Key_Left:
            self.rudder += 5
            print(self.rudder)
            self.rudder_x = math.cos(math.radians(self.rudder))*200
            self.rudder_y = -math.sin(math.radians(self.rudder))*200
                
    def fire(self, rad):
        b = Bullet(rad)
        #총알이 만들어 졌을 당시에 target의 현재위치를 총알에 저장함
        b.target_state = self.target.get_state()
        #그 총알을 총알리스트에 담아준다
        self.bullets.append(b)
        #총알을 몇번 발사했는지 확인하는 변수 fireCnt
        self.fireCnt += 1
        
    def reset(self): # 에피소드가 끝나고 새로 에피소드를 시작할때
        self.target.reset()
        self.fireCnt = 0
        self.hits = 0
        self.bullets.clear() # bullets안에 있는 총알들을 전부 없앰
        self.episodes = 400  # 
        return self.target.get_state() # Target의 현재위치를 return 해준다
        
    def step(self, action, rad):
        self.episodes -= 1
        done = False
        if self.episodes == 0:
            done = True
        self.rad = rad
        
        if self.start==3:
            action,rad = self.action_auto()
        
        if action==1 : #action 1일경우(발사)에만 fire함수를 돌려줌
            self.fire(rad)
        self.move()
        #현재상태, 보상, 끝났는지 여부, 딕셔너리
        return self.target.get_state(),0,done,{}

In [5]:
# Thread를 이용한 Window 실행
# QMainWindow를 실행할 때는 QApplication 인스턴스의 exec_()함수를 실행해야 한다
# exec_()함수는 이벤트 루프(무한루프)를 실행하므로 exec_() 아래에 있는 코드는 실행되지 않는다
# 그러므로 QMainWindow 와 함께 다른 코드를 실행하려면 아래와 같이 한개의 Thread로 실행하면 된다

import threading

#나중에 아래 코드에서 window를 또 사용할 일이 생기기 때문에 전역변수로 선언해줌.
window = None

def start_window():
    app = QApplication(sys.argv)
    global window
    window = MainWindow()
    window.show()
    app.exec_()

win_thread = threading.Thread(target=start_window)
win_thread.start()

In [4]:
# 포물선 운동 애니메이션
# 이동 애니메이션

import sys
import time
import math
import gym
import numpy as np
from PyQt5.QtCore import *
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

# 발사 여부를 계산할 신경망 생성
# 뉴런의 수와 은닉층의 수는 테스트를 진행하면서 변경해가면서 적정수를 찾는다

def create_fire_model():
    fire_model = Sequential()
    fire_model.add(Dense(64, activation='relu', input_shape=(2,)))
    fire_model.add(Dense(64, activation='relu'))
    fire_model.add(Dense(64, activation='relu'))
    fire_model.add(Dense(64, activation='relu'))
    fire_model.add(Dense(2, activation="softmax"))

    fire_model.compile(
        loss="categorical_crossentropy",
        optimizer="adam",
        metrics=["accuracy"])
    return fire_model

# 발사 각도(rad)를 계산할 신경망 생성
def create_rad_model():
    rad_model = Sequential()
    rad_model.add(Dense(64, activation='relu', input_shape=(2,)))
    rad_model.add(Dense(64, activation='relu'))
    rad_model.add(Dense(64, activation='relu'))
    rad_model.add(Dense(64, activation='relu'))
    rad_model.add(Dense(1, activation="linear"))

    rad_model.compile(
        loss="mse",
        optimizer="adam",
        metrics=["mae"])
    return rad_model

# 파일에 저장된 모델의 가중치를 로드하여 모델에 적용
action_model = create_fire_model()  # 모델생성 및 컴파일
action_model.load_weights('action_model_weights.h5')

rad_model = create_rad_model()
rad_model.load_weights('rad_model_weights.h5')

class Target:
    def __init__(self):
        self.x = 0
        self.y = 300
    def move(self):
        self.x += 1
        self.y = int((1/132) * (self.x-200)**2 + 0) 
    def get_state(self):
        return self.x, self.y
    def reset(self):
        self.x = 0
        self.y = 300
    
class Bullet:
    def __init__(self, rad):
        self.rad = rad
        self.x = 195
        self.y = 290
        self.vx = math.cos(rad)*3
        self.vy = -math.sin(rad)*3    # sin이 -인 이유는 y좌표는 위로가야하기 때문에
        self.target_state = (0,0)
    def move(self):
        self.x += self.vx
        self.y += self.vy

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.target = Target()
        self.bullets = []
        self.label = QLabel()
        self.label.setAlignment(Qt.AlignCenter)
        self.label.setPixmap(QPixmap(400, 300))
        self.ishit = False   # 맞췄을때
        # Target이 0에서 오른쪽으로 400이동해야 하기때문에 에피소드 횟수를 400으로 함
        self.episodes = 400  
        self.fireCnt = 0
        self.hits = 0
        self.hit_state = []
        self.hit_rad = []
        self.timer = QTimer()
        self.start = 0
        self.rudder = 90
        self.skip = 10
        self.rudder_x = 0
        self.rudder_y = -280
        
        move_action = QAction('&Start', self)
        #move_action.setStatusTip("Click to start")
        move_action.triggered.connect(self.action_start)
        
        stop_action = QAction('Stop', self)
        stop_action.triggered.connect(self.action_stop)
        
        autosemi_action = QAction('Semi-auto',self)
        autosemi_action.triggered.connect(self.semiauto)
        
        auto_action = QAction('&Auto',self)
        auto_action.triggered.connect(self.action_auto)
        
        toolbar = QToolBar()
        self.addToolBar(toolbar)
        
        toolbar.addAction(move_action)
        toolbar.addAction(autosemi_action)
        toolbar.addAction(auto_action)
        toolbar.addAction(stop_action)

        self.setCentralWidget(self.label)
    
    def paintEvent(self, event):
        painter = QPainter(self.label.pixmap())
        painter.fillRect(QRect(0,0,400,300), Qt.black)  # 화면 지우기
        
        if self.start==1 :
            pen = QPen(Qt.white, 3, Qt.CustomDashLine)
            pen.setDashPattern([4, 3, 2, 5])
            painter.setPen(pen)
            painter.drawLine(200, 295, (200 + int(self.rudder_x)), (280 + int(self.rudder_y)) )

        painter.setPen(QPen(Qt.white,  2, Qt.SolidLine))  # 테두리 선
        brush = None
        if self.ishit:
            brush = QBrush(Qt.white, Qt.SolidPattern) # 폭발시 흰색으로 타겟을 그림
            self.ishit = False
        else: brush = QBrush(Qt.red, Qt.SolidPattern)
        painter.setBrush(brush)  # 채울 색상, 패턴
        # drawEllipse(QRect), QRect(x, y, w, h)
        painter.drawEllipse(self.target.x, self.target.y, 20, 20)
        #Target의 x,y좌표에 원 그리기
        
        for b in self.bullets:  #총알을 하나하나 꺼낸다
            painter.setPen(Qt.white)
            painter.setBrush(QBrush(Qt.white, Qt.SolidPattern))
            painter.drawEllipse(int(b.x), int(b.y), 10, 10)
        painter.end()
        
    def set_models(self,actionM,radM):
        self.action_model = actionM
        self.rad_model = radM
    
    def action_moves(self) :
        for e in range(10) :
            self.action_move()
        
    def action_move(self):
        self.timer = QTimer()
        self.timer.setInterval(30)
        self.timer.timeout.connect(self.move)
        self.timer.start()

    def action_start(self):
        self.start = 1
        
    def semiauto(self):
        self.start = 2
        
    def action_auto(self):
        self.start= 3
        done = False
        self.episodes -= 1
        if self.episodes == 0:
            done = True
        self.rad = rad
        while not done:
            state = self.target.get_state()
            prob = action_model.predict(np.array(state).reshape(-1,2))
            action = np.argmax(prob[0])
            self.skip -= 1
            print(action)
            if self.skip!=0 :
                action = 0
            else :
                self.skip=10
            print(action)
            rad = rad_model.predict(np.array(state).reshape(-1,2))
            return action,rad

    def action_stop(self):
        self.timer.stop()
        
    def move(self):
        self.target.move()
        for b in self.bullets:
            b.move()
            #포탄의 y좌표가 천장에 닿았을때 or x좌표가 왼쪽,오른쪽에 닿앗을때
            if b.y<=-10 or b.x<=0 or b.x>=400:
                self.bullets.remove(b)
            # 타겟과 발사체의 충돌(명중) 검사
            # 피타고라스 정의를 사용
            if ((self.target.x-b.x)**2 + (self.target.y-b.y)**2) < (20**2) :
                #명중 했을때의 타겟위치의 상태
                self.hit_state.append(b.target_state)
                #명중 했을때의 타겟위치의 총알의 시작각도
                self.hit_rad.append(b.rad)
                try:
                    #명중시 그 탄환 삭제 위에서 remove된 총알을 또 삭제할려고 할
                    #때 오류가 발생함으로
                    # 그 오류를 그냥 pass하기 위해 try except를 사용함
                    # 또는 elif를 사용하면 문제가 사라짐
                    self.bullets.remove(b)
                except:
                    pass
                # ishit 맞췄을시에 Target을 흰색으로 표시하기 위한 변수
                self.ishit = True 
                self.hits += 1
        #좌표가 바뀔때 마다 화면을 다시그리기 위한 함수 repaint()
        self.repaint()

    def keyPressEvent(self, evt):
        if evt.key()==Qt.Key_Space:
            if self.start==1 :
                rad = math.radians(self.rudder)
                self.step(1,rad)
                
            elif self.start==2:
                while True:
                    state = self.target.get_state()
                    prob = action_model.predict(np.array(state).reshape(-1,2))
                    if np.argmax(prob[0])==1: #쏠때가 아니라면 쏘지 않는다
                        rad = rad_model.predict(np.array(state).reshape(-1,2))
                        self.step(1,rad)
                        break
        if evt.key() ==Qt.Key_Right:
            self.rudder -= 5
            print(self.rudder)                                                              # 탄환이 움직이는 각도
            self.rudder_x = math.cos(math.radians(self.rudder))*200
            self.rudder_y = -math.sin(math.radians(self.rudder))*200
#             self.rudder_y = -math.sin(3.14/2)*200    # 선 움직이기
        if evt.key()==Qt.Key_Left:
            self.rudder += 5
            print(self.rudder)
            self.rudder_x = math.cos(math.radians(self.rudder))*200
            self.rudder_y = -math.sin(math.radians(self.rudder))*200
            
    def fire(self, rad):
        b = Bullet(rad)
        #총알이 만들어 졌을 당시에 target의 현재위치를 총알에 저장함
        b.target_state = self.target.get_state()
        #그 총알을 총알리스트에 담아준다
        self.bullets.append(b)
        #총알을 몇번 발사했는지 확인하는 변수 fireCnt
        self.fireCnt += 1
        
    def reset(self): # 에피소드가 끝나고 새로 에피소드를 시작할때
        self.target.reset()
        self.fireCnt = 0
        self.hits = 0
        self.bullets.clear() # bullets안에 있는 총알들을 전부 없앰
        self.episodes = 400  # 
        return self.target.get_state() # Target의 현재위치를 return 해준다
        
    def step(self, action, rad):
        self.episodes -= 1
        done = False
        if self.episodes == 0:
            done = True
        self.rad = rad
        
        #if self.start==3:
        #    action,rad = self.action_move()
        
        if action==1 : #action 1일경우(발사)에만 fire함수를 돌려줌
            self.fire(rad)
        self.move()
        #현재상태, 보상, 끝났는지 여부, 딕셔너리
        return self.target.get_state(),0,done,{}

In [6]:
# Test
import time

window.set_models(action_model, rad_model)

for e in range(10):
    state = window.reset()
    done = False
    while not done:
        _,_,done,_ = window.step(0,0)
        time.sleep(0.02)
    print('Episode %s fires %s hits %s' % (e+1, window.fireCnt, window.hits) )
print('테스트 종료')

85
80
75
70
65
60
55
Episode 1 fires 9 hits 5
50
45
40
35
30
25
20
15
10
5
0
Episode 2 fires 13 hits 2
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
120
115
110
105
100
95
90
85
80
75
70
65
60
55
50
45
40
35
30
25
20
15
10
5
Episode 3 fires 28 hits 6


RuntimeError: wrapped C/C++ object of type MainWindow has been deleted

In [ ]:
# QMainWindow 외부에서 윈도우 제어가 가능한지 테스트
# 타겟이 포물선 운동을 할 때 발사체를 임의의 각도로 발사하여 우연히 명중되는 빈도수를 확인한다
import random
import time

print('무작위 테스트 시작')

for e in range(3):
    state = window.reset() # 현재 Target의 위치 0,300 부터 시작
    done = False
    while not done:
        rad = math.radians(np.random.randint(0,180))  # 무작위 발사 각
        action = 1
        state,reward,done,info = window.step(1,rad)   # action=1 으로 발사 요청
        time.sleep(0.02)
    print('Episode %s fires %s hits %s' % (e+1, window.fireCnt, window.hits) )
print('무작위 테스트 종료')

In [ ]:
# 신경망 학습을 위해 게임으로부터 데이터 수집
# window.hit_state : 명중한 발사체로부터 수집된 타겟 상태(x, y)
# window.hit_rad : 명중한 발사체로부터 수집된 발사 각도(radian)

import time

print('무작위 대응으로 게임 데이터 수집')
states = []
# 데이터가 많을수록 학습효과와 정확도가 높다
for e in range(300):
    state = window.reset()
    done = False
    while not done:
        rad = math.radians(np.random.randint(0,180))  # 무작위 발사 각
        action = 1
        state,reward,done,info = window.step(1,rad)   # action=1 으로 발사 요청
        states.append(state)
        #time.sleep(0.02)
    print('Episode %s fires %s hits %s' % (e+1, window.fireCnt, window.hits) )
print('데이터 수집 종료')

#print(window.hit_state)
#print(window.hit_rad)

In [ ]:
# 신경망에 사용할 수 있도록 데이터 처리
# 발사여부, 발사각을 계산할 수 있는 신경망 2개를 사용할 목적으로 처리
# 발사여부를 계산할 신경망을 학습할 문제(x,y), 정답(0 or 1)

import numpy as np
fire_label = []

for s in states:
    ohe = np.zeros(2)               # 원핫 인코딩용 
    if s in window.hit_state:       # 맞췄을때의 상태가 현재 상태와 같을때
        ohe[1] = 1                  # 1을 넣어준다
    else: ohe[0] = 1
    fire_label.append(ohe)

stateX = np.array(states)           #모든 상태정보를 ndarray로 만듬
fireY = np.array(fire_label)        # 0과 1로 이루어진 list를 ndarray로 만듬

print(type(stateX), stateX.shape)
print(type(fireY), fireY.shape)

In [ ]:
# 발사각 계산용 신경망을 학습시킬 데이터 준비( 문제(x,y), 정답(rad))
#총알이 타겟에 맞춘 상태정보를 문제로 넣기위해 만든 변수
hit_stateX = np.array(window.hit_state)    #맞췄을때의 상태
#스칼라 데이터를 np.array로 만듬
hit_radY = np.array(window.hit_rad).reshape(-1,1) # 맞췄을때의 rad각도

print(type(hit_stateX), hit_stateX.shape)
print(type(hit_radY), hit_radY.shape)

In [ ]:
# 발사 여부를 계산할 신경망 생성

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

# 뉴런의 수와 은닉층의 수는 테스트를 진행하면서 변경해가면서 적정수를 찾는다
def create_fire_model():
    fire_model = Sequential()
    fire_model.add(Dense(64, activation='relu', input_shape=(2,)))
    fire_model.add(Dense(64, activation='relu'))
    fire_model.add(Dense(64, activation='relu'))
    fire_model.add(Dense(64, activation='relu'))
    fire_model.add(Dense(2, activation="softmax"))

    fire_model.compile(
        loss="categorical_crossentropy",
        optimizer="adam",
        metrics=["accuracy"])
    return fire_model

In [ ]:
# 발사 각도(rad)를 계산할 신경망 생성

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

def create_rad_model():
    rad_model = Sequential()
    rad_model.add(Dense(64, activation='relu', input_shape=(2,)))
    rad_model.add(Dense(64, activation='relu'))
    rad_model.add(Dense(64, activation='relu'))
    rad_model.add(Dense(64, activation='relu'))
    rad_model.add(Dense(1, activation="linear"))

    rad_model.compile(
        loss="mse",
        optimizer="adam",
        metrics=["mae"])
    return rad_model

In [ ]:
action_model = create_fire_model()
action_model.fit(stateX, fireY, epochs=100)

In [ ]:
rad_model = create_rad_model()
rad_model.fit(hit_stateX, hit_radY, epochs=200)

In [ ]:
# 학습된 fire_model, rad_model을 사용한 발사여부, 발사각 계산 및 적용
import time

skip = 10
for e in range(10):
    state = window.reset()
    done = False
    while not done:
        #rad = math.radians(np.random.randint(0,180))  # 무작위 발사 각
        rad = rad_model.predict(np.array(state).reshape(-1,2))
        prob = action_model.predict(np.array(state).reshape(-1,2))

        action = np.argmax(prob[0])
        
        skip -= 1    # 10 프레임마다 한번씩 발사될 가능성 설정
        if skip!=0:
             action=0
        else:
             skip = 10
        state,reward,done,info = window.step(action,rad) 
        #time.sleep(0.02)
    print('Episode %s fires %s hits %s' % (e+1, window.fireCnt, window.hits) )
print('테스트 종료')

In [ ]:
# 학습된 모델의 가중치(Weight)를 파일에 저장
action_model.save('action_model_weights.h5')
rad_model.save('rad_model_weights.h5')

In [ ]:
# 파일에 저장된 모델의 가중치를 로드하여 모델에 적용
action_model = create_fire_model()  # 모델생성 및 컴파일
action_model.load_weights('action_model_weights.h5')

rad_model = create_rad_model()
rad_model.load_weights('rad_model_weights.h5')

In [ ]:
# 가중치를 파일에서 로드하여 발사여부, 발사각 계산 및 적용
import time


skip = 15
for e in range(10):
    state = window.reset()
    done = False
    while not done:
        rad = rad_model.predict(np.array(state).reshape(-1,2))
        prob = action_model.predict(np.array(state).reshape(-1,2))

        action = np.argmax(prob[0])
        
        skip -= 1    # 10 프레임마다 한번씩 발사될 가능성 설정
        if skip!=0:
            action=0
        else:
            skip = 15
        state,reward,done,info = window.step(action,rad) 
        time.sleep(0.02)
    print('Episode %s fires %s hits %s' % (e+1, window.fireCnt, window.hits) )
print('테스트 종료')

In [ ]:
# 이용자가 스페이스키를 누를 때만 발사되어 타겟에 명중하도록 하기
# 가중치를 파일에서 로드하여 발사여부, 발사각 계산 및 적용
import time

window.set_models(action_model, rad_model)

for e in range(10):
    state = window.reset()
    done = False
    while not done:
        _,_,done,_ = window.step(0,0)
        time.sleep(0.02)
    print('Episode %s fires %s hits %s' % (e+1, window.fireCnt, window.hits) )
print('테스트 종료')

In [ ]:
math.radians(5)